## Part 1 - Training the Neural Network

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf


In [39]:
yf.download('USDJPY=X', start = '2020-01-01', end = '2023-12-31', progress = False).to_csv('USDJPY.csv')

In [40]:
df = pd.read_csv('USDJPY.csv', index_col = 'Date', parse_dates = True)
data = df.copy()
data.head()


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-01,108.680000,111.410004,108.629997,108.681000,108.681000,0
2020-01-02,108.713997,108.860001,108.221001,108.712997,108.712997,0
2020-01-03,108.540001,108.551003,107.915001,108.542999,108.542999,0
2020-01-06,107.999001,108.410004,107.921997,107.963997,107.963997,0
2020-01-07,108.411003,108.616997,108.261002,108.405998,108.405998,0


In [41]:
# Checking the Shape
print(data.shape)

# 1043 observations

(1043, 6)


## Feature Engineering

In [42]:
import talib as ta

In [43]:
# Lets build a Feature List. We can include the following features:
# 1. Rolling Mean
# 2. Rolling Standard Deviation
#3. Momentum
# 4. Intraday Movement
# 5. Bollinger Bands
# 6. MACD
# 7. SAR



feature_list = []

# 1. Rolling Mean - Trend
for i in range(5, 20 , 5):
    col_name = 'Rolling_Mean_'+str(i)
    data[col_name] = data['Close'].rolling(window = i).mean()
    feature_list.append(col_name)
    
# 2. Rolling Standard Deviation - Volatility
for i in range(10,30,5):
    col_name = 'Rolling_STD_'+str(i)
    data[col_name] = data['Close'].rolling(window=i).std()
    feature_list.append(col_name)
    
# 3. Cummulative sum of pct_change - Momentum
for i in range(3,12,3):
    col_name = 'Momentum_'+str(i)
    data[col_name] = data['Close'].pct_change().rolling(window=i).sum()
    feature_list.append(col_name)
    
# 4. Intraday Movement
col_name = 'CO'
data['CO'] = data['Close']- data['Open']
feature_list.append('CO')

# 5. Bollinger Bands
data['upper_band'], data['middle_band'], data['lower_band'] = ta.BBANDS(data['Close'].values, timeperiod = 20)
feature_list.append('upper_band')
feature_list.append('middle_band')
feature_list.append('lower_band')

# 6. MACD
data['MACD'], data['Signal_Line'], data['MACD_Histogram'] = ta.MACD(data['Close'].values, fastperiod = 12, slowperiod = 26, signalperiod = 9)
feature_list.append('MACD')

# 7. SAR
data['SAR'] = ta.SAR(data['High'].values, data['Low'].values, acceleration = 0.02, maximum = 0.2)
feature_list.append('SAR')

# 8. RSI
data['RSI'] = ta.RSI(data['Close'].values, timeperiod = 14)
feature_list.append('RSI')
    
    

In [44]:
feature_list

['Rolling_Mean_5',
 'Rolling_Mean_10',
 'Rolling_Mean_15',
 'Rolling_STD_10',
 'Rolling_STD_15',
 'Rolling_STD_20',
 'Rolling_STD_25',
 'Momentum_3',
 'Momentum_6',
 'Momentum_9',
 'CO',
 'upper_band',
 'middle_band',
 'lower_band',
 'MACD',
 'SAR',
 'RSI']

In [45]:
data[feature_list].head()

,Rolling_Mean_5,Rolling_Mean_10,Rolling_Mean_15,Rolling_STD_10,Rolling_STD_15,Rolling_STD_20,Rolling_STD_25,Momentum_3,Momentum_6,Momentum_9,CO,upper_band,middle_band,lower_band,MACD,SAR,RSI
Date,,,,,,,,,,,,,,,,,
2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000999,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000999,NaN,NaN,NaN,NaN,111.410004,NaN
2020-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002998,NaN,NaN,NaN,NaN,111.346224,NaN
2020-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.006604,NaN,NaN,-0.035004,NaN,NaN,NaN,NaN,111.208975,NaN
2020-01-07,108.461398,NaN,NaN,NaN,NaN,NaN,NaN,-0.002804,NaN,NaN,-0.005005,NaN,NaN,NaN,NaN,111.077216,NaN


In [46]:
data.isna().sum()

Open                0
High                0
Low                 0
Close               0
Adj Close           0
Volume              0
Rolling_Mean_5      4
Rolling_Mean_10     9
Rolling_Mean_15    14
Rolling_STD_10      9
Rolling_STD_15     14
Rolling_STD_20     19
Rolling_STD_25     24
Momentum_3          3
Momentum_6          6
Momentum_9          9
CO                  0
upper_band         19
middle_band        19
lower_band         19
MACD               33
Signal_Line        33
MACD_Histogram     33
SAR                 1
RSI                14
dtype: int64

In [47]:
# Remove Nan Values since the ML Algorihms cannot handle Nan Values
data.dropna(inplace = True)

In [48]:
data.isna().sum()

Open               0
High               0
Low                0
Close              0
Adj Close          0
Volume             0
Rolling_Mean_5     0
Rolling_Mean_10    0
Rolling_Mean_15    0
Rolling_STD_10     0
Rolling_STD_15     0
Rolling_STD_20     0
Rolling_STD_25     0
Momentum_3         0
Momentum_6         0
Momentum_9         0
CO                 0
upper_band         0
middle_band        0
lower_band         0
MACD               0
Signal_Line        0
MACD_Histogram     0
SAR                0
RSI                0
dtype: int64

In [49]:
## Assign feature list to X and target to y
X = data[feature_list]

data['target'] = np.where(data['Close'].shift(-1) > data['Close'],1,-1)
y = data['target']

In [50]:
## Split the data into Train and Test
from sklearn import model_selection
X_train,X_test,y_train,y_test = model_selection.train_test_split(X,y,test_size = 0.2, shuffle= False)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((808, 17), (202, 17), (808,), (202,))

In [51]:
# Before we train the model, we need to scale the data
from sklearn import preprocessing
scalar = preprocessing.StandardScaler()
X_train_scaled = scalar.fit_transform(X_train)
X_test_scaled = scalar.transform(X_test)


In [52]:
X_train_scaled_df = pd.DataFrame(X_train_scaled,columns = X_train.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns = X_test.columns)

In [55]:

X_train_scaled_df.describe().round(2)# You can see the mean and STD are scaled to 0 and 1

,Rolling_Mean_5,Rolling_Mean_10,Rolling_Mean_15,Rolling_STD_10,Rolling_STD_15,Rolling_STD_20,Rolling_STD_25,Momentum_3,Momentum_6,Momentum_9,CO,upper_band,middle_band,lower_band,MACD,SAR,RSI
count,808.00,808.00,808.00,808.00,808.00,808.00,808.00,808.00,808.00,808.00,808.00,808.00,808.00,808.00,808.00,808.00,808.00
mean,0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00
std,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
min,-1.09,-1.07,-1.07,-1.19,-1.14,-1.17,-1.12,-4.38,-4.40,-4.02,-7.17,-1.08,-1.06,-1.05,-2.93,-1.22,-2.49
25%,-0.77,-0.77,-0.76,-0.73,-0.75,-0.76,-0.79,-0.51,-0.47,-0.50,-0.18,-0.74,-0.75,-0.76,-0.43,-0.76,-0.69
50%,-0.52,-0.53,-0.53,-0.37,-0.38,-0.44,-0.48,-0.00,0.00,-0.02,-0.01,-0.51,-0.53,-0.48,-0.16,-0.49,-0.14
75%,0.95,0.95,0.95,0.51,0.48,0.61,0.71,0.57,0.49,0.50,0.16,0.93,0.97,0.99,0.46,0.94,0.72
max,2.44,2.39,2.38,5.00,4.16,3.41,3.34,4.69,4.36,3.93,22.51,2.37,2.37,2.55,2.43,2.66,2.80


In [60]:
# We need to use MLP Classifier (Multi Layer Perceptron) to train the model
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes = (8,8,8), max_iter = 1000,activation = 'tanh',solver = 'adam',shuffle = False, random_state = 1)
mlp.fit(X_train_scaled_df,y_train)

MLPClassifier(activation='tanh', hidden_layer_sizes=(8, 8, 8), max_iter=1000,
              random_state=1, shuffle=False)

In [61]:
mlp.n_layers_

5

In [62]:
mlp.get_params()

{'activation': 'tanh',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (8, 8, 8),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 1000,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': 1,
 'shuffle': False,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

In [65]:
# Check weights
mlp.coefs_[0].shape, mlp.coefs_[1].shape, mlp.coefs_[2].shape, mlp.coefs_[3].shape

((17, 8), (8, 8), (8, 8), (8, 1))

In [66]:
mlp.intercepts_[0].shape, mlp.intercepts_[1].shape, mlp.intercepts_[2].shape, mlp.intercepts_[3].shape

((8,), (8,), (8,), (1,))

In [68]:
# Check for model accurancy on Train and Test
train_score = mlp.score(X_train_scaled_df,y_train)
test_score = mlp.score(X_test_scaled_df,y_test)
print('Train Score:',train_score)
print('Test Score:',test_score)


Train Score: 0.7054455445544554
Test Score: 0.5297029702970297


In [69]:
# Predictions
y_predict = mlp.predict(X_test_scaled_df)
y_predict

array([ 1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1, -1,  1,  1,  1, -1,
       -1,  1,  1,  1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1,  1, -1, -1,
       -1,  1, -1, -1, -1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1, -1, -1,  1, -1, -1,  1,  1,  1,  1,
        1,  1,  1, -1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1,  1, -1, -1,
       -1, -1,  1,  1, -1, -1, -1,  1,  1, -1, -1, -1, -1, -1,  1,  1,  1,
        1, -1, -1, -1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,
        1, -1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1, -1, -1,  1, -1,  1, -1,  1,  1,  1, -1, -1,  1, -1,
       -1, -1,  1,  1, -1, -1,  1,  1,  1, -1, -1, -1,  1,  1,  1,  1, -1,
       -1, -1, -1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1,
       -1, -1, -1,  1,  1,  1, -1, -1, -1, -1, -1,  1,  1,  1,  1])

In [70]:
# Prceision, Recall and F1 Score
from sklearn import metrics
precision = metrics.precision_score(y_test,y_predict)
recall = metrics.recall_score(y_test,y_predict)
f1_score = metrics.f1_score(y_test,y_predict)
print('Precision:',precision)
print('Recall:',recall)
print('F1 Score:',f1_score)

Precision: 0.568
Recall: 0.6339285714285714
F1 Score: 0.5991561181434599


## Part 2 - Backtesting the model

So far we've covered
* Read Data
* Create Features
* Scale data
* Use already trained model to make predictions
* Trade on those prediction, and calculate the strategy returns

In [79]:
data['Predicted_Signal'] = y_predict

ValueError: Length of values (202) does not match length of index (1010)